# Market Basket Analysis on Retail Transactions

## Objective
The objective of this notebook is to analyze retail transaction data and
identify frequently co-purchased products using Market Basket Analysis.
Association rule mining techniques such as Apriori / FP-Growth will be used
to extract meaningful patterns.


## Import Required Libraries


In [43]:
import pandas as pd
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
pio.templates.default = "plotly_white"
import numpy as np
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.frequent_patterns import association_rules


## Load the Dataset

The dataset is loaded using a semicolon (`;`) as the delimiter, which is commonly
used in European-style CSV files. Loading the dataset correctly ensures that
columns are parsed accurately.


In [15]:
data = pd.read_csv(
    "market_basket_dataset.csv",
    sep=";"
)
# The warning in this cell output is because there are mixed types of bill numbers like 536365 and C536379
# So pandas is not sure what data type to assign to the 'BillNo' column
# But we can ignore this warning for now as it does not affect our analysis and we wont do any operations that depend on the data type of 'BillNo' column

C:\Users\athar\AppData\Local\Temp\ipykernel_53064\936468460.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(


### Preview of the Dataset

In [12]:
data.head()


,BillNo,Itemname,Quantity,Date,Price,CustomerID,Country
0,536365,WHITE HANGING HEART T-LIGHT HOLDER,6,01.12.2010 08:26,"2,55",17850.0,United Kingdom
1,536365,WHITE METAL LANTERN,6,01.12.2010 08:26,"3,39",17850.0,United Kingdom
2,536365,CREAM CUPID HEARTS COAT HANGER,8,01.12.2010 08:26,"2,75",17850.0,United Kingdom
3,536365,KNITTED UNION FLAG HOT WATER BOTTLE,6,01.12.2010 08:26,"3,39",17850.0,United Kingdom
4,536365,RED WOOLLY HOTTIE WHITE HEART.,6,01.12.2010 08:26,"3,39",17850.0,United Kingdom


## Initial Data Inspection

This step helps us understand the structure of the dataset, including:
- Number of rows and columns
- Data types of each column
- Presence of missing values
- Memory usage

This understanding is essential before performing any cleaning or transformation.


In [16]:
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 522064 entries, 0 to 522063
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   BillNo      522064 non-null  object 
 1   Itemname    520609 non-null  object 
 2   Quantity    522064 non-null  int64  
 3   Date        522064 non-null  object 
 4   Price       522064 non-null  object 
 5   CustomerID  388023 non-null  float64
 6   Country     522064 non-null  object 
dtypes: float64(1), int64(1), object(5)
memory usage: 27.9+ MB


### Observations from Initial Inspection
(After running data.info(), write observations like below in Markdown)
- The dataset contains 522,064 rows and 7 columns.
- `Price` is numeric (`float64`).
- `Quantity` is integer (`int64`).
- `Itemname` and `CustomerID` contain missing values.
- `BillNo` contains mixed types and is stored as an object.
- `Date` is currently stored as an object and not as a datetime.


## Data Understanding

In this section, we describe what each column in the dataset represents.
Understanding the meaning of each column helps determine which columns are
relevant for Market Basket Analysis and which can be ignored or used later.

### Column Descriptions

| Column Name | Description |
|------------|-------------|
| BillNo | Unique transaction or invoice identifier |
| Itemname | Name of the product purchased |
| Quantity | Number of units purchased in the transaction |
| Date | Date of the transaction |
| Price | Price per unit of the product |
| CustomerID | Unique customer identifier |
| Country | Country where the transaction occurred |


### Relevance for Market Basket Analysis

For Market Basket Analysis, we are primarily interested in:
- `BillNo` – to define a transaction
- `Itemname` – to define products
- `Quantity` – to identify whether an item was purchased

Other columns such as `Price`, `Date`, `CustomerID`, and `Country` are not
required for association rule mining at this stage and will be ignored for now.


## Data Cleaning – Handling Missing Values

Market Basket Analysis requires complete information about transactions and
items. Transactions with missing product names cannot contribute to meaningful
association rules, so such rows must be removed.


### Missing Values Overview


In [17]:
data.isnull().sum()


BillNo             0
Itemname        1455
Quantity           0
Date               0
Price              0
CustomerID    134041
Country            0
dtype: int64

### Decision on Missing Values

- Rows with missing `Itemname` are removed because the product is unknown.
- `CustomerID` contains missing values, but it is not required for Market Basket
  Analysis and will be ignored.
- No rows are missing `BillNo` or `Quantity`, so those columns are safe.


In [18]:
data = data.dropna(subset=["Itemname"])

### Verification After Removing Missing Values


In [20]:
data.isnull().sum()


BillNo             0
Itemname           0
Quantity           0
Date               0
Price              0
CustomerID    132586
Country            0
dtype: int64

## Selecting Relevant Columns

For Market Basket Analysis, we only require information about:
- The transaction identifier
- The product purchased
- The quantity purchased

All other columns are not needed for association rule mining at this stage.
Selecting only the relevant columns simplifies the dataset and improves
processing efficiency.


In [22]:
data = data[["BillNo", "Itemname", "Quantity"]]


### Verification After Column Selection


In [23]:
data.head()


,BillNo,Itemname,Quantity
0,536365,WHITE HANGING HEART T-LIGHT HOLDER,6
1,536365,WHITE METAL LANTERN,6
2,536365,CREAM CUPID HEARTS COAT HANGER,8
3,536365,KNITTED UNION FLAG HOT WATER BOTTLE,6
4,536365,RED WOOLLY HOTTIE WHITE HEART.,6


In [24]:
data.info()


<class 'pandas.core.frame.DataFrame'>
Index: 520609 entries, 0 to 522063
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   BillNo    520609 non-null  object
 1   Itemname  520609 non-null  object
 2   Quantity  520609 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 15.9+ MB


## Aggregating Items Within Each Transaction

A single transaction (BillNo) may contain the same product multiple times.
Before creating the market basket matrix, we need to aggregate quantities
for each product within each transaction.


In [25]:
transaction_item = (
    data
    .groupby(["BillNo", "Itemname"])["Quantity"]
    .sum()
    .reset_index()
)


### Verification After Aggregation


In [26]:
transaction_item.head()


,BillNo,Itemname,Quantity
0,536365,CREAM CUPID HEARTS COAT HANGER,8
1,536365,GLASS STAR FROSTED T-LIGHT HOLDER,6
2,536365,KNITTED UNION FLAG HOT WATER BOTTLE,6
3,536365,RED WOOLLY HOTTIE WHITE HEART.,6
4,536365,SET 7 BABUSHKA NESTING BOXES,2


In [27]:
transaction_item.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 509829 entries, 0 to 509828
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   BillNo    509829 non-null  object
 1   Itemname  509829 non-null  object
 2   Quantity  509829 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 11.7+ MB


## Creating the Transaction–Item Matrix

Market Basket Analysis algorithms require data in the form of a matrix where:
- Each row represents a transaction
- Each column represents a product
- Each cell indicates the quantity of that product purchased in the transaction


In [28]:
basket = (
    transaction_item
    .pivot(index="BillNo", columns="Itemname", values="Quantity")
    .fillna(0)
)


### Verification of Basket Matrix


In [29]:
basket.head()


Itemname,*Boombox Ipod Classic,*USB Office Mirror Ball,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 DAISY PEGS IN WOOD BOX,12 EGG HOUSE PAINTED WOOD,12 HANGING EGGS HAND PAINTED,12 IVORY ROSE PEG PLACE SETTINGS,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,...,wrongly coded 20713,wrongly coded 23343,wrongly coded-23343,wrongly marked,wrongly marked 23343,wrongly marked carton 22804,wrongly marked. 23343 in box,wrongly sold (22719) barcode,wrongly sold as sets,wrongly sold sets
BillNo,,,,,,,,,,,,,,,,,,,,,
536365,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536366,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536367,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536368,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536369,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
basket.shape


(20210, 4185)

## Binary Encoding of the Basket Matrix

Market Basket Analysis algorithms such as Apriori and FP-Growth require binary
input, where:
- 1 indicates the presence of an item in a transaction
- 0 indicates the absence of an item

The actual quantity purchased is not required for association rule mining.


In [31]:
basket_binary = basket.map(lambda x: 1 if x > 0 else 0)


C:\Users\athar\AppData\Local\Temp\ipykernel_53064\2222279361.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  basket_binary = basket.applymap(lambda x: 1 if x > 0 else 0)


## Converting Basket Matrix to Boolean Format

FP-Growth requires the transaction–item matrix to be in boolean format,
where True indicates the presence of an item and False indicates its absence.


In [38]:
basket_bool = basket_binary.astype(bool)


In [39]:
basket_bool.dtypes.value_counts()


bool    4185
Name: count, dtype: int64

## Applying FP-Growth Algorithm

FP-Growth is used to efficiently discover frequent itemsets from large
transactional datasets without generating candidate itemsets.


In [41]:
frequent_itemsets_fp = fpgrowth(
    basket_bool,
    min_support=0.01,
    use_colnames=True
)
frequent_itemsets_fp.head()


,support,itemsets
0,0.108956,(WHITE HANGING HEART T-LIGHT HOLDER)
1,0.022613,(KNITTED UNION FLAG HOT WATER BOTTLE)
2,0.020881,(RED WOOLLY HOTTIE WHITE HEART.)
3,0.018555,(SET 7 BABUSHKA NESTING BOXES)
4,0.014795,(WHITE METAL LANTERN)


In [42]:
frequent_itemsets_fp.sort_values("support", ascending=False).head(10)


,support,itemsets
0,0.108956,(WHITE HANGING HEART T-LIGHT HOLDER)
92,0.102128,(JUMBO BAG RED RETROSPOT)
294,0.094211,(REGENCY CAKESTAND 3 TIER)
576,0.081940,(PARTY BUNTING)
42,0.076249,(LUNCH BAG RED RETROSPOT)
7,0.070807,(ASSORTED COLOUR BIRD ORNAMENT)
606,0.066601,(SET OF 3 CAKE TINS PANTRY DESIGN)
43,0.063286,(PACK OF 72 RETROSPOT CAKE CASES)
172,0.062345,(LUNCH BAG BLACK SKULL.)
79,0.060960,(NATURAL SLATE HEART CHALKBOARD)


## Generating Association Rules

After identifying frequent itemsets using FP-Growth, the next step is to
generate association rules. Association rules help us understand relationships
between products in the form:

IF item(s) A are purchased → THEN item(s) B are likely to be purchased.

The strength of these rules is measured using metrics such as:
- Support
- Confidence
- Lift


### Creating Association Rules

- Confidence measures how often items in B appear in transactions that contain A.
- Lift measures how much more often A and B occur together than expected by chance.


In [44]:
rules = association_rules(
    frequent_itemsets_fp,
    metric="confidence",
    min_threshold=0.3
)


### Preview of Association Rules


In [45]:
rules.head()


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
0,(HAND WARMER OWL DESIGN),(HAND WARMER UNION JACK),0.032113,0.023701,0.011677,0.363636,15.342570,1.0,0.010916,1.534184,0.965838,0.264574,0.348188,0.428165
1,(HAND WARMER UNION JACK),(HAND WARMER OWL DESIGN),0.023701,0.032113,0.011677,0.492693,15.342570,1.0,0.010916,1.907893,0.957516,0.264574,0.475862,0.428165
2,(HAND WARMER UNION JACK),(HAND WARMER SCOTTY DOG DESIGN),0.023701,0.026967,0.010638,0.448852,16.644577,1.0,0.009999,1.765465,0.962738,0.265760,0.433577,0.421674
3,(HAND WARMER SCOTTY DOG DESIGN),(HAND WARMER UNION JACK),0.026967,0.023701,0.010638,0.394495,16.644577,1.0,0.009999,1.612372,0.965970,0.265760,0.379796,0.421674
4,(HOME BUILDING BLOCK WORD),(WHITE HANGING HEART T-LIGHT HOLDER),0.037803,0.108956,0.012370,0.327225,3.003279,1.0,0.008251,1.324431,0.693237,0.092047,0.244959,0.220379


In [48]:
rules.sort_values("lift", ascending=False).head(10)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
780,(HERB MARKER THYME),(HERB MARKER ROSEMARY),0.011529,0.011578,0.010737,0.931330,80.436704,1.0,0.010604,14.393889,0.999086,0.868000,0.930526,0.929340
779,(HERB MARKER ROSEMARY),(HERB MARKER THYME),0.011578,0.011529,0.010737,0.927350,80.436704,1.0,0.010604,13.606013,0.999136,0.868000,0.926503,0.929340
785,(HERB MARKER THYME),(HERB MARKER PARSLEY),0.011529,0.011479,0.010391,0.901288,78.513024,1.0,0.010259,10.014143,0.998778,0.823529,0.900141,0.903230
786,(HERB MARKER PARSLEY),(HERB MARKER THYME),0.011479,0.011529,0.010391,0.905172,78.513024,1.0,0.010259,10.423877,0.998728,0.823529,0.904066,0.903230
788,(HERB MARKER PARSLEY),(HERB MARKER ROSEMARY),0.011479,0.011578,0.010391,0.905172,78.177498,1.0,0.010258,10.423355,0.998673,0.820312,0.904062,0.901304
787,(HERB MARKER ROSEMARY),(HERB MARKER PARSLEY),0.011578,0.011479,0.010391,0.897436,78.177498,1.0,0.010258,9.638075,0.998773,0.820312,0.896245,0.901304
790,(HERB MARKER MINT),(HERB MARKER PARSLEY),0.011677,0.011479,0.010292,0.881356,76.776739,1.0,0.010158,8.331816,0.998637,0.800000,0.879978,0.888954
789,(HERB MARKER PARSLEY),(HERB MARKER MINT),0.011479,0.011677,0.010292,0.896552,76.776739,1.0,0.010158,9.553785,0.998437,0.800000,0.895329,0.888954
775,(HERB MARKER ROSEMARY),(HERB MARKER BASIL),0.011578,0.011677,0.010341,0.893162,76.486491,1.0,0.010206,9.250700,0.998487,0.800766,0.891900,0.889378
776,(HERB MARKER BASIL),(HERB MARKER ROSEMARY),0.011677,0.011578,0.010341,0.885593,76.486491,1.0,0.010206,8.639537,0.998587,0.800766,0.884253,0.889378


## Filtering and Interpreting Association Rules

Not all generated rules are useful. In this section, we filter the association
rules based on meaningful thresholds and interpret them from a business
perspective.


### Filtering Rules Based on Strength

We focus on rules that satisfy:
- High confidence (reliability of the rule)
- Lift greater than 1 (positive association)


In [49]:
strong_rules = rules[
    (rules["confidence"] >= 0.5) &
    (rules["lift"] > 1)
]


### Strong Association Rules


In [51]:
strong_rules.sort_values("lift", ascending=False)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
780,(HERB MARKER THYME),(HERB MARKER ROSEMARY),0.011529,0.011578,0.010737,0.931330,80.436704,1.0,0.010604,14.393889,0.999086,0.868000,0.930526,0.929340
779,(HERB MARKER ROSEMARY),(HERB MARKER THYME),0.011578,0.011529,0.010737,0.927350,80.436704,1.0,0.010604,13.606013,0.999136,0.868000,0.926503,0.929340
785,(HERB MARKER THYME),(HERB MARKER PARSLEY),0.011529,0.011479,0.010391,0.901288,78.513024,1.0,0.010259,10.014143,0.998778,0.823529,0.900141,0.903230
786,(HERB MARKER PARSLEY),(HERB MARKER THYME),0.011479,0.011529,0.010391,0.905172,78.513024,1.0,0.010259,10.423877,0.998728,0.823529,0.904066,0.903230
788,(HERB MARKER PARSLEY),(HERB MARKER ROSEMARY),0.011479,0.011578,0.010391,0.905172,78.177498,1.0,0.010258,10.423355,0.998673,0.820312,0.904062,0.901304
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1279,(PINK REGENCY TEACUP AND SAUCER),(REGENCY CAKESTAND 3 TIER),0.036418,0.094211,0.018308,0.502717,5.336092,1.0,0.014877,1.821478,0.843308,0.162996,0.450995,0.348523
1158,(SUKI SHOULDER BAG),(JUMBO BAG RED RETROSPOT),0.026076,0.102128,0.013904,0.533207,5.220984,1.0,0.011241,1.923491,0.830111,0.121645,0.480112,0.334675
382,(TOY TIDY PINK POLKADOT),(JUMBO BAG RED RETROSPOT),0.023058,0.102128,0.012073,0.523605,5.126967,1.0,0.009718,1.884723,0.823951,0.106737,0.469418,0.320911
1209,(RED RETROSPOT SHOPPER BAG),(JUMBO BAG RED RETROSPOT),0.028204,0.102128,0.014201,0.503509,4.930190,1.0,0.011320,1.808435,0.820304,0.122284,0.447036,0.321280


In [56]:
frequent_itemsets_fp.to_csv(
    "frequent_itemsets_fp.csv",
    index=False
)


In [55]:
rules.to_csv(
    "association_rules_all.csv",
    index=False
)


In [54]:
strong_rules.to_csv(
    "association_rules_strong.csv",
    index=False
)
